### 학습셋, 테스트 셋 만들기 

In [ ]:
import os
#os.environ['MXNET_ENGINE_TYPE'] = 'NaiveEngine'
#os.environ['MXNET_CUDNN_AUTOTUNE_DEFAULT'] = '1'

In [1]:
import re
import numpy as np
from tqdm import tqdm
#mecab = Mecab()
from konlpy.tag import Mecab
from stemming.porter2 import stem
mecab = Mecab()


In [2]:

##어근 추출 
def make_list(fname, l= 'ko'):
    spaced = []
    with open(fname) as f:
        if l == 'ko':
            spaced = [['START', ] + mecab.morphs(i.strip()) + ['END', ] for i in f.readlines()]
        else:
            spaced = [['START', ] + [stem(j) for j in i.lower().split()] + ['END', ] for i in f.readlines()]
    return(spaced)


train_ko = make_list('../korean_parallel_corpora/korean-english-v1/korean-english-park.train.ko',l='ko')
len(train_ko)
test_ko = make_list('../korean_parallel_corpora/korean-english-v1/korean-english-park.test.ko',l='ko')
len(test_ko)


train_en = make_list('../korean_parallel_corpora/korean-english-v1/korean-english-park.train.en',l='en')
len(train_en)
test_en = make_list('../korean_parallel_corpora/korean-english-v1/korean-english-park.test.en',l='en')
len(test_en)

2000

In [3]:
max_seq_len = 30

In [4]:
#한글 문자 영어 문장 두 문장 쌍 중 하나의 문장이라도 max_seq_len을 초과하는지 확인 후 제거 
#완벽한 문장으로 학습을 하기 위함 

tr_idx_ko = [idx  for idx, i in enumerate(train_ko) if len(i) <= max_seq_len]
tr_idx_en = [idx  for idx, i in enumerate(train_en) if len(i) <= max_seq_len]

tr = np.intersect1d(tr_idx_ko, tr_idx_en)
tr = set(tr)

te_idx_ko = [idx  for idx, i in enumerate(test_ko) if len(i) <= max_seq_len]
te_idx_en = [idx  for idx, i in enumerate(test_en) if len(i) <= max_seq_len]

te = np.intersect1d(te_idx_ko, te_idx_en)
te = set(te)

train_ko_f = [i  for idx, i in enumerate(train_ko) if idx in tr] 
train_en_f = [i  for idx, i in enumerate(train_en) if idx in tr] 

test_ko_f = [i  for idx, i in enumerate(test_ko) if idx in te] 
test_en_f = [i  for idx, i in enumerate(test_en) if idx in te] 

In [9]:
from  embedding_maker import *
w2idx, idx_2w = load_vocab("../ko_en.dic") 

In [10]:
max_seq = max_seq_len

In [11]:
def pad_sequences(sequences, maxlen=None, dtype='int32',
                  padding='pre', truncating='pre', value=0.):
    if not hasattr(sequences, '__len__'):
        raise ValueError('`sequences` must be iterable.')
    lengths = []
    for x in sequences:
        if not hasattr(x, '__len__'):
            raise ValueError('`sequences` must be a list of iterables. '
                             'Found non-iterable: ' + str(x))
        lengths.append(len(x))

    num_samples = len(sequences)
    if maxlen is None:
        maxlen = np.max(lengths)

    # take the sample shape from the first non empty sequence
    # checking for consistency in the main loop below.
    sample_shape = tuple()
    for s in sequences:
        if len(s) > 0:
            sample_shape = np.asarray(s).shape[1:]
            break

    x = (np.ones((num_samples, maxlen) + sample_shape) * value).astype(dtype)
    for idx, s in enumerate(sequences):
        if not len(s):
            continue  # empty list/array was found
        if truncating == 'pre':
            trunc = s[-maxlen:]
        elif truncating == 'post':
            trunc = s[:maxlen]
        else:
            raise ValueError('Truncating type "%s" not understood' % truncating)

        # check `trunc` has expected shape
        trunc = np.asarray(trunc, dtype=dtype)
        if trunc.shape[1:] != sample_shape:
            raise ValueError('Shape of sample %s of sequence at position %s is different from expected shape %s' %
                             (trunc.shape[1:], idx, sample_shape))

        if padding == 'post':
            x[idx, :len(trunc)] = trunc
        elif padding == 'pre':
            x[idx, -len(trunc):] = trunc
        else:
            raise ValueError('Padding type "%s" not understood' % padding)
    return x


In [12]:
def encoding_and_padding(corp_list, dic, max_seq=50):
    coding_seq = [ [dic.get(j, dic['__ETC__']) for j in i]  for i in corp_list ]
    return(pad_sequences(coding_seq, maxlen=max_seq, padding='post', truncating='post',value=dic['__PAD__']))
    

In [13]:
ko_train_x = encoding_and_padding(train_ko_f, w2idx, max_seq=max_seq)
ko_test_x = encoding_and_padding(test_ko_f, w2idx, max_seq=max_seq)


In [14]:
en_train_x = encoding_and_padding(train_en_f, w2idx, max_seq=max_seq)
en_test_x = encoding_and_padding(test_en_f, w2idx, max_seq=max_seq)

In [15]:
train_en_lag = [ i[1:] for i in train_en_f]
test_en_lag = [ i[1:] for i in test_en_f]


In [16]:
en_train_y = encoding_and_padding(train_en_lag, w2idx, max_seq=max_seq)
en_test_y = encoding_and_padding(test_en_lag, w2idx, max_seq=max_seq)

In [18]:
max_seq_length = max_seq 
vocab_size = len(w2idx)
n_hidden = 256
embed_dim = 50
embed_weights  = load_embedding("../ko_en.np")
embed_weights.shape

(21002, 50)

#### Gluon Modeling 

In [19]:
import numpy as np
import pandas as pd
from mxnet import nd as  F
import mxnet as mx
from  mxnet import gluon
from mxnet.gluon import nn, rnn
import mxnet.autograd as autograd
from mxnet import nd as F


In [38]:
class korean_english_translator(gluon.HybridBlock):
    
    def __init__(self, n_hidden, vocab_size, embed_dim, max_seq_length,attention=False ,**kwargs):
        super(korean_english_translator,self).__init__(**kwargs)
        #입력 시퀀스 길이
        self.in_seq_len = max_seq_length
        #출력 시퀀스 길이 
        self.out_seq_len = max_seq_length
        # LSTM의 hidden 개수 
        self.n_hidden = n_hidden
        #고유문자개수 
        self.vocab_size = vocab_size
        #max_seq_length
        self.max_seq_length = max_seq_length
        #임베딩 차원수 
        self.embed_dim = embed_dim
        
        
        self.attention = attention
        with self.name_scope():
            self.embedding = nn.Embedding(input_dim=vocab_size, output_dim=embed_dim, dtype="float16")

            
            self.encoder= rnn.BidirectionalCell(rnn.GRUCell(hidden_size=int(n_hidden/2)),  
                                                rnn.GRUCell(hidden_size=int(n_hidden/2)))
            self.decoder = rnn.GRUCell(hidden_size=n_hidden)
            self.batchnorm = nn.BatchNorm(axis=2)
            #flatten을 false로 할 경우 마지막 차원에 fully connected가 적용된다. 
            #(N, max_seq_len,  vacab size)
            self.dense = nn.Dense(self.vocab_size,flatten=False)
            if self.attention:
                self.attdense = nn.Dense(self.max_seq_length, flatten=False, activation='relu')
                self.attn_combine = nn.Dense( self.embed_dim, flatten=False, activation='relu')
        #self.embedding.weight.set_data(embed_weights)
        #self.embedding.collect_params().setattr('grad_req', 'null')
            
    def hybrid_forward(self, F, inputs, outputs):
        """
        학습 코드 
        """
        #encoder LSTM
        embeddinged_in = F.cast(self.embedding(inputs), dtype='float32')

        enout, (next_l, next_r) = self.encoder.unroll(inputs=embeddinged_in, length=self.in_seq_len, 
                                                          merge_outputs=True)   
        next_h = F.concat(next_l, next_r, dim=1)
        embeddinged_out = F.cast(self.embedding(outputs),dtype='float32')
        
        #decoder LSTM with attention
        for i in range(self.out_seq_len):
            #out_seq_len 길이만큼 LSTMcell을 unroll하면서 출력값을 적재한다. 
            p_outputs = F.slice_axis(embeddinged_out, axis=1, begin=i, end=i+1)
            p_outputs = F.reshape(p_outputs, (-1, self.embed_dim))
            # p_outputs = embeddinged_out[:,i,:]
            # 위와 같이 진행한 이유는 hybridize를 위함 
            if self.attention:
                print(p_outputs.shape)
                p_outputs, _ = self.apply_attention(F=F, inputs=p_outputs, hidden=next_h, encoder_outputs=enout)
                print(p_outputs.shape)
            deout, (next_h,) = self.decoder(p_outputs, [next_h,] )
            if i == 0:
                deouts = deout
            else:
                deouts = F.concat(deouts, deout, dim=1)
        #2dim -> 3dim 으로 reshape 
        deouts_orig = F.reshape(deouts, (-1, self.out_seq_len, self.n_hidden))
        
        deouts = self.batchnorm(deouts_orig)
        deouts_fc = self.dense(deouts)
        return(deouts_fc)
    
    def apply_attention(self, F, inputs, hidden, encoder_outputs):
        #inputs : decoder input의미
        concated = F.concat(inputs, hidden, dim=1)
        #(,max_seq_length) : max_seq_length 개별 시퀀스의 중요도  
        attn_weights = F.softmax(self.attdense(concated), axis=1)
        # (N,max_seq_length,n_hidden) x (N,max_seq_length,1) = (N, max_seq_length, n_hidden)
        #attn_weigths 가중치를 인코더 출력값에 곱해줌
        w_encoder_outputs = F.broadcast_mul(encoder_outputs, attn_weights.expand_dims(2))
        #(N, embed_dim), (N, max_seq_length * n_hidden) = (N, ...)
        output = F.concat(inputs.flatten(), w_encoder_outputs.flatten(), dim=1)
        #(N, vocab_size)
        output = self.attn_combine(output)
        #attention weight은 시각화를 위해 뽑아둔다. 
        return(output, attn_weights)
    
    def calulation(self, input_str, ko_dict, en_dict, en_rev_dict, ctx=mx.gpu(0)):
        """
        inference 코드 
        """
        #앞뒤에 START,END 코드 추가 
        input_str = [['START', ] + mecab.morphs(input_str.strip()) + ['END', ],]
        X = encoding_and_padding(input_str, ko_dict, max_seq=self.max_seq_length)
        #string to embed 
        X = F.cast(self.embedding(F.array(X, ctx=ctx)), dtype='float32')

        #인코더 출력값을 도출한다. 
        #enout_base, (next_h,) = self.encoder.unroll(inputs=X, length=self.in_seq_len, merge_outputs=True)
        enout, (next_l, next_r) = self.encoder.unroll(inputs=X, length=self.in_seq_len, 
                                                          merge_outputs=True)
        next_h = F.concat(next_l, next_r, dim=1)
        #디코더의 초기 입력값으로 넣을 'START'를 임베딩한다.
        Y_init = F.array([[en_dict['START'],],], ctx=ctx)
        Y_init = F.cast(self.embedding(Y_init),dtype='float32')
        deout = Y_init[:,0,:]
        
        #출력 시퀀스 길이만큼 순회 
        for i in range(self.out_seq_len):
            if self.attention:
                #print(deout.shape)
                deout, att_weight = self.apply_attention(F=F, inputs=deout, hidden=next_h, encoder_outputs=enout)
                if i == 0:
                    att_weights = att_weight
                else:
                    att_weights = F.concat(att_weights,att_weight,dim=0)
            deout, (next_h, ) = self.decoder(deout, [next_h, ])
            #batchnorm을 적용하기 위해 차원 증가/원복 
            deout = F.expand_dims(deout,axis=1)
            deout = self.batchnorm(deout)
            #reduce dim
            deout = deout[:,0,:]
            #'START'의 다음 시퀀스 출력값도출 
            deout_sm = self.dense(deout)
            #print(deout_sm.shape)
            deout = F.one_hot(F.argmax(F.softmax(deout_sm, axis=1), axis=1), depth=self.vocab_size)
            #print(deout.shape)
            #decoder에 들어갈 수 있는 형태로 변환(임베딩 적용 및 차원 맞춤)
            deout = F.argmax(deout, axis=1)
            deout = F.expand_dims(deout, axis=0)
            deout = F.cast(self.embedding(deout)[:,0,:],dtype='float32')
            gen_char  = en_rev_dict[F.argmax(deout_sm, axis=1).asnumpy()[0].astype('int')]
            if gen_char == '__PAD__' or gen_char == 'END':
                break
            else: 
                if i == 0:
                    ret_seq = [gen_char, ]
                else:
                    ret_seq += [gen_char, ]
        return(" ".join(ret_seq), att_weights)

                

### 모델링 




In [39]:
tr_set = gluon.data.ArrayDataset(ko_train_x, en_train_x, en_train_y)
tr_data_iterator = gluon.data.DataLoader(tr_set, batch_size=100, shuffle=True)

te_set =gluon.data.ArrayDataset(ko_test_x, en_test_x, en_test_y)
te_data_iterator = gluon.data.DataLoader(te_set, batch_size=100, shuffle=True)

for l, (i,j,k) in enumerate(tr_data_iterator):
    i = i
    j = j
    k = k
    break

x = F.random.normal(0, 1, shape=(1,30))
y = F.random.normal(0, 1, shape=(1,30))


model = korean_english_translator(n_hidden, vocab_size, embed_dim, max_seq_length,attention=True)

model.collect_params().initialize(mx.init.Xavier(), ctx=mx.cpu(0))

model(x,y)

In [ ]:
GPU_COUNT = 2
ctx= [mx.gpu(i) for i in range(GPU_COUNT)]

def model_init():
    #모형 인스턴스 생성 및 트래이너, loss 정의 
    #n_hidden, vocab_size, embed_dim, max_seq_length
    model = korean_english_translator(n_hidden, vocab_size, embed_dim, max_seq_length,attention=True)
    model.collect_params().initialize(mx.init.Xavier(), ctx=ctx)
    model.embedding.weight.set_data(embed_weights)
         
    trainer = gluon.Trainer(model.collect_params(), 'rmsprop', kvstore='local') 
    loss = gluon.loss.SoftmaxCrossEntropyLoss(axis = 2)
    return(model, loss, trainer)

In [ ]:
def calculate_loss(model, data_iter, loss_obj, ctx=ctx):
    test_loss = []
    for i, (x_data, y_data, z_data) in enumerate(data_iter):
        x_data_l = gluon.utils.split_and_load(x_data, ctx,  even_split=False)
        y_data_l = gluon.utils.split_and_load(y_data, ctx,  even_split=False)
        z_data_l = gluon.utils.split_and_load(z_data, ctx,  even_split=False)
        with autograd.predict_mode():
            losses = [loss_obj(model(x, y), z) for x, y, z in zip(x_data_l, y_data_l, z_data_l)]
        curr_loss = (mx.nd.mean( losses[0] ).asscalar() + mx.nd.mean( losses[1] ).asscalar())/2
        test_loss.append(curr_loss)
    return(np.mean(test_loss))

In [ ]:
import time

def train(epochs, tr_data_iterator, model, loss, trainer, ctx= [mx.gpu(i) for i in range(GPU_COUNT)], mdl_desc="k2e_model"):
    ### 학습 코드 
    tot_test_loss = []
    tot_train_loss = []
    for e in range(epochs):
        tic = time.time()
        # Decay learning rate.
        if e > 1:
            trainer.set_learning_rate(trainer.learning_rate * 0.5)
        train_loss = []
        for i, (x_data, y_data, z_data) in enumerate(tqdm(tr_data_iterator)):
            #x_data = x_data.as_in_context(ctx).astype('float32')
            #y_data = y_data.as_in_context(ctx).astype('float32')
            #z_data = z_data.as_in_context(ctx).astype('float32')
            
            x_data_l = gluon.utils.split_and_load(x_data, ctx, even_split=False)
            y_data_l = gluon.utils.split_and_load(y_data, ctx, even_split=False)
            z_data_l = gluon.utils.split_and_load(z_data, ctx, even_split=False)
            
            with autograd.record():
                losses = [loss(model(x, y), z) for x, y, z in zip(x_data_l, y_data_l, z_data_l)]
            for l in losses:
                l.backward()
            trainer.step(x_data.shape[0])
            curr_loss = (mx.nd.mean( losses[0] ).asscalar() + mx.nd.mean( losses[1] ).asscalar())/2
            train_loss.append(curr_loss)
            mx.nd.waitall()
        #caculate test loss
        test_loss = calculate_loss(model, te_data_iterator, loss_obj = loss, ctx=ctx) 
        print('[Epoch %d] time cost: %f'%(e, time.time()-tic))
        print("Epoch %s. Train Loss: %s, Test Loss : %s" % (e, np.mean(train_loss), test_loss))    
        tot_test_loss.append(test_loss)
        tot_train_loss.append(np.mean(train_loss))
        model.save_params("{}_{}.params".format(mdl_desc, e))
    return(tot_test_loss, tot_train_loss)

In [ ]:
model, loss, trainer = model_init()

In [ ]:
print(model)

In [ ]:
%%time
model.hybridize()
train(10, tr_data_iterator, model, loss, trainer)

In [ ]:
#retrain 
trainer = gluon.Trainer(model.collect_params(), 'sgd', optimizer_params={'learning_rate':1,'clip_gradient':0.99, 'wd':1e-5,})
#loss = gluon.loss.SoftmaxCrossEntropyLoss(axis = 2)
tr_set = gluon.data.ArrayDataset(ko_train_x, en_train_x, en_train_y)
tr_data_iterator = gluon.data.DataLoader(tr_set, batch_size=100, shuffle=True)


In [ ]:
%%time
#epochs = 5
#model.hybridize()
test_loss, train_loss = train(10, tr_data_iterator, model, loss, trainer,ctx=ctx,mdl_desc='k2e_model_sgd')

In [ ]:
eng_seq,att_weight  = model.calulation("좋은 아침.", ko_dict=w2idx, en_dict=w2idx, en_rev_dict=idx_2w)

eng_seq